In [2]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
rating=[]
votes = []
description = []
director_list = []
star_list = []
imdb_id = []

#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,10000,50)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else " "
        time.append(runtime)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else " "
        rating.append(rate)
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text if store.find_all('span', attrs = {'name': "nv"}) else " "
        votes.append(vote)
        
        # Description of the Movies 
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) > 1 else ' '
        description.append(description_)
        
        ## Director 
        ps = store.find_all('p')
        for p in ps:
            if 'Director'in p.text:
                director =p.find('a').text if p.find('a') else ' '
        
        director_list.append(director)
        
        ## actors
        star = store.find("p", attrs={"class":""}).text.replace("Stars:", "").replace("\n", "").replace("Director:", "").replace("Directors:", "")
        star_list.append(star)
        
        # IMDB ID
        imdbID = store.find('span','rating-cancel').a.get('href').split('/')[2] if store.find('span','rating-cancel') else ' '
        imdb_id.append(imdbID)

''

### Making the scrapping data into a DataFrame

In [3]:
movie_list = pd.DataFrame({"imdb-id": imdb_id, "movie_name": movie_name, "year_of_release": year, "runtime" : time, "IMDB_rating" : rating,
                           "no_of_votes": votes, "plot_description" : description, "director": director_list, "actors":star_list})

In [5]:
movie_list.head()

,imdb-id,movie_name,year_of_release,runtime,IMDB_rating,no_of_votes,plot_description,director,actors
0,tt10811166,The Kashmir Files,(2022),170 min,8.3,"548,031","The Kashmir Files' is a story, based on video ...",Vivek Agnihotri,"Vivek Agnihotri| Mithun Chakraborty, A..."
1,tt1187043,3 Idiots,(2009),170 min,8.4,"387,020",Two friends are searching for their long lost ...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Madhavan,..."
2,tt0986264,Taare Zameen Par,(2007),165 min,8.3,"188,938",An eight-year-old boy is thought to be a lazy ...,Aamir Khan,"Aamir Khan, Amole Gupte| Darsheel Safa..."
3,tt5074352,Dangal,(2016),161 min,8.3,"183,452",Former wrestler Mahavir Singh Phogat and his t...,Nitesh Tiwari,"Nitesh Tiwari| Aamir Khan, Sakshi Tanw..."
4,tt2338151,PK,(2014),153 min,8.1,"180,108",An alien on Earth loses the only device he can...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Anushka S..."


### Creating a csv file

In [6]:
movie_list.to_csv("bollywood_data_set.csv")

In [7]:
df = pd.read_csv("bollywood_data_set.csv")

### Cleaning the dataset

In [8]:
df = df.drop("Unnamed: 0", axis = 1)

In [9]:
df['imdb-id'] = df['imdb-id'].str.replace("tt", '')

In [12]:
df['year_of_release'] = df['year_of_release'].str.replace("(", "")
df['year_of_release'] = df['year_of_release'].str.replace(")", "")

C:\Users\musta\AppData\Local\Temp/ipykernel_2572/981502636.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['year_of_release'] = df['year_of_release'].str.replace("(", "")
C:\Users\musta\AppData\Local\Temp/ipykernel_2572/981502636.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['year_of_release'] = df['year_of_release'].str.replace(")", "")


In [13]:
df.head()

,imdb-id,movie_name,year_of_release,runtime,IMDB_rating,no_of_votes,plot_description,director,actors
0,10811166,The Kashmir Files,2022,170 min,8.3,"548,031","The Kashmir Files' is a story, based on video ...",Vivek Agnihotri,"Vivek Agnihotri| Mithun Chakraborty, A..."
1,1187043,3 Idiots,2009,170 min,8.4,"387,020",Two friends are searching for their long lost ...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Madhavan,..."
2,0986264,Taare Zameen Par,2007,165 min,8.3,"188,938",An eight-year-old boy is thought to be a lazy ...,Aamir Khan,"Aamir Khan, Amole Gupte| Darsheel Safa..."
3,5074352,Dangal,2016,161 min,8.3,"183,452",Former wrestler Mahavir Singh Phogat and his t...,Nitesh Tiwari,"Nitesh Tiwari| Aamir Khan, Sakshi Tanw..."
4,2338151,PK,2014,153 min,8.1,"180,108",An alien on Earth loses the only device he can...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Anushka S..."
